In [117]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()
with open("../login.json", "r") as j:
    misDatos = json.load(j)
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [118]:
nuevos = "C:/Users/jmartinez/Documents/reportes_metlife2.xlsx"
Pacientes_Archivo = pd.read_excel(nuevos)
Pacientes_Archivo.head(5)

,Codigo Ficha,Rut,Nombre,Sexo,Campaña,Holding,Empleador,Poliza_x,Fecha,Fecha_chilena,...,Meses,Canal,Clínica,Motivo de consulta,Nombre Empresa,Corredor,Pais,Rut_Titular_y,Relación_y,Index
0,FSA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,Noviembre,LLamada,Orientación Médica,NaN,Prueba MetLife,NaN,Chile,169397759,Titular,1
1,SA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,Noviembre,Agendamiento,Orientación Médica,NaN,Prueba MetLife,NaN,Chile,169397759,Titular,2
2,FSA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,Noviembre,LLamada,Deporte,NaN,Clínica Alemana,Willis Towers Watson S.A. Corred. De Seg.,Chile,106879508,Titular,3
3,SA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,Noviembre,Agendamiento,Orientación Médica,NaN,Clínica Alemana,Willis Towers Watson S.A. Corred. De Seg.,Chile,106879508,Titular,4
4,FSA-0059,248270454,Aaron Antonio Aguayo Perez,Hombre,MetLife,Alsea,Starbucks Coffee Chile S.A.,340020238,2022-11-21T16:00:00.000+0000,2022-11-21 16:00:00,...,Noviembre,LLamada,Deporte,NaN,Starbucks Coffee Chile S.A.,Aon Risk Services (Chile) Corred. De Seg.,Chile,191841344,Carga,5


In [119]:
df3 = pd.DataFrame()
Pacientes_Archivo.sort_values("Fecha_chilena", inplace=True)
Pacientes_Archivo.Rut_Titular_y.fillna(Pacientes_Archivo["Rut"], inplace=True)

Pacientes_Archivo_moviles = Pacientes_Archivo[(Pacientes_Archivo["Fecha"] >= "2023-02-01") & (Pacientes_Archivo["Fecha"] <= "2024-01-31")]
Pacientes_Archivo_moviles["llave"] = "Enero" + Pacientes_Archivo["Rut_Titular_y"].astype(str)

Pacientes_Archivo.head(5)
Pacientes_Archivo["llave"] = Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Rut_Titular_y"].astype(str)
#Pacientes_Archivo["llaveEnero"] = "Enero"+ Pacientes_Archivo["Rut_Titular_y"].astype(str)

Pacientes_Archivo["Familia_Mensual"] = Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Mes"].astype(str) + Pacientes_Archivo["Rut_Titular_y"].astype(str)
Pacientes_Archivo["rut_Unico_Mensual"] =  Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Mes"].astype(str) + Pacientes_Archivo["Rut"].astype(str)
Pacientes_Archivo["rut_Unico_Anual"] =  Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Rut"].astype(str)
Pacientes_Archivo["uso_titulares"] = np.where(Pacientes_Archivo["Rut_Titular_y"] == Pacientes_Archivo["Rut"], Pacientes_Archivo["Rut"].astype(str), 0)
Pacientes_Archivo["uso_titulares_Mes"] = np.where(Pacientes_Archivo["Rut_Titular_y"] == Pacientes_Archivo["Rut"], Pacientes_Archivo["Rut"].astype(str) + Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Mes"].astype(str) , 0)
Pacientes_Archivo["uso_titulares_Año"] = np.where(Pacientes_Archivo["Rut_Titular_y"] == Pacientes_Archivo["Rut"], Pacientes_Archivo["Rut"].astype(str) + Pacientes_Archivo["Año"].astype(str) , 0)
#Pacientes_Archivo["hora_Agenda"] = Pacientes_Archivo.Fecha_chilena.hour


Pacientes_Archivo.head()


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_8892\2193912708.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pacientes_Archivo_moviles["llave"] = "Enero" + Pacientes_Archivo["Rut_Titular_y"].astype(str)


,Codigo Ficha,Rut,Nombre,Sexo,Campaña,Holding,Empleador,Poliza_x,Fecha,Fecha_chilena,...,Rut_Titular_y,Relación_y,Index,llave,Familia_Mensual,rut_Unico_Mensual,rut_Unico_Anual,uso_titulares,uso_titulares_Mes,uso_titulares_Año
0,FSA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,169397759,Titular,1,2022169397759,202211169397759,202211169397759,2022169397759,169397759,169397759202211,1693977592022
1,SA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,169397759,Titular,2,2022169397759,202211169397759,202211169397759,2022169397759,169397759,169397759202211,1693977592022
2,FSA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,106879508,Titular,3,2022106879508,202211106879508,202211106879508,2022106879508,106879508,106879508202211,1068795082022
3,SA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,106879508,Titular,4,2022106879508,202211106879508,202211106879508,2022106879508,106879508,106879508202211,1068795082022
4,FSA-0059,248270454,Aaron Antonio Aguayo Perez,Hombre,MetLife,Alsea,Starbucks Coffee Chile S.A.,340020238,2022-11-21T16:00:00.000+0000,2022-11-21 16:00:00,...,191841344,Carga,5,2022191841344,202211191841344,202211248270454,2022248270454,0,0,0


In [120]:
df3 = Pacientes_Archivo_moviles[[ "Index","llave"]]
df3 = df3.drop_duplicates(
    [
        "llave",
    ],
    keep="first",
)
df3["año_movil"] = 1 
df3 = df3[[ "Index","año_movil"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.año_movil.fillna(0, inplace=True)
df3.head()

,Index,año_movil
15448,15449,1
15449,15450,1
15450,15451,1
15451,15452,1
15453,15454,1


In [121]:
Pacientes_Archivo.head()

,Codigo Ficha,Rut,Nombre,Sexo,Campaña,Holding,Empleador,Poliza_x,Fecha,Fecha_chilena,...,Relación_y,Index,llave,Familia_Mensual,rut_Unico_Mensual,rut_Unico_Anual,uso_titulares,uso_titulares_Mes,uso_titulares_Año,año_movil
0,FSA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,Titular,1,2022169397759,202211169397759,202211169397759,2022169397759,169397759,169397759202211,1693977592022,0.0
1,SA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,Titular,2,2022169397759,202211169397759,202211169397759,2022169397759,169397759,169397759202211,1693977592022,0.0
2,FSA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,Titular,3,2022106879508,202211106879508,202211106879508,2022106879508,106879508,106879508202211,1068795082022,0.0
3,SA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,Titular,4,2022106879508,202211106879508,202211106879508,2022106879508,106879508,106879508202211,1068795082022,0.0
4,FSA-0059,248270454,Aaron Antonio Aguayo Perez,Hombre,MetLife,Alsea,Starbucks Coffee Chile S.A.,340020238,2022-11-21T16:00:00.000+0000,2022-11-21 16:00:00,...,Carga,5,2022191841344,202211191841344,202211248270454,2022248270454,0,0,0,0.0


In [122]:
df3 = Pacientes_Archivo[[ "Index","llave"]]
df3 = df3.drop_duplicates(
    [
        "llave",
    ],
    keep="first",
)
df3["año_movil_Calenario"] = 1 
df3 = df3[[ "Index","año_movil_Calenario"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.año_movil_Calenario.fillna(0, inplace=True)


In [123]:
df3 = Pacientes_Archivo[[ "Index","Familia_Mensual"]]
df3 = df3.drop_duplicates(
    [
        "Familia_Mensual",
    ],
    keep="first",
)
df3["Familia_Mensual_CT"] = 1 
df3 = df3[[ "Index","Familia_Mensual_CT"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.Familia_Mensual_CT.fillna(0, inplace=True)
df3.head()

,Index,Familia_Mensual_CT
0,1,1
2,3,1
4,5,1
6,7,1
8,9,1


In [124]:
#["rut Unico Mensual"]
df3 = Pacientes_Archivo[[ "Index","rut_Unico_Mensual"]]
df3 = df3.drop_duplicates(
    [
        "rut_Unico_Mensual",
    ],
    keep="first",
)
df3["rut_Unico_Mensual_CT"] = 1 
df3 = df3[[ "Index","rut_Unico_Mensual_CT"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.rut_Unico_Mensual_CT.fillna(0, inplace=True)
df3.head()

,Index,rut_Unico_Mensual_CT
0,1,1
2,3,1
4,5,1
6,7,1
8,9,1


In [125]:
Pacientes_Archivo.head(5)


,Codigo Ficha,Rut,Nombre,Sexo,Campaña,Holding,Empleador,Poliza_x,Fecha,Fecha_chilena,...,Familia_Mensual,rut_Unico_Mensual,rut_Unico_Anual,uso_titulares,uso_titulares_Mes,uso_titulares_Año,año_movil,año_movil_Calenario,Familia_Mensual_CT,rut_Unico_Mensual_CT
0,FSA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,202211169397759,202211169397759,2022169397759,169397759,169397759202211,1693977592022,0.0,1.0,1.0,1.0
1,SA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,202211169397759,202211169397759,2022169397759,169397759,169397759202211,1693977592022,0.0,0.0,0.0,0.0
2,FSA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,202211106879508,202211106879508,2022106879508,106879508,106879508202211,1068795082022,0.0,1.0,1.0,1.0
3,SA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,202211106879508,202211106879508,2022106879508,106879508,106879508202211,1068795082022,0.0,0.0,0.0,0.0
4,FSA-0059,248270454,Aaron Antonio Aguayo Perez,Hombre,MetLife,Alsea,Starbucks Coffee Chile S.A.,340020238,2022-11-21T16:00:00.000+0000,2022-11-21 16:00:00,...,202211191841344,202211248270454,2022248270454,0,0,0,0.0,1.0,1.0,1.0


In [126]:
from pivottablejs import pivot_ui

In [129]:
Pacientes_Archivo.to_excel("C:/Users/jmartinez/Documents/reportes_2111.xlsx", index=False)

In [128]:


#Pacientes_Archivo.to_excel("C:/Users/jmartinez/Documents/reportes_2111.xlsx", index=False)
pivot_ui(Pacientes_Archivo)